In [24]:
using LinearAlgebra
# SVD algorithm
# Takes matrix A as input, outputs unitary matrices U, V and diagonal matrix S
# Such that A = U*S*V^T

#Implementing SVD algorithm
#cutNum is the constant k that you want to have best k-rank approximation
function mySVD(A, cutNum=-1)
    m, n = size(A)
    tall = true
    
    if (n > m)
        tall = false
        A = A'
        m, n = size(A)
    end

    # compute singular values and vectors for V
    sigma, singVec_V = eigen(A' * A)
    # does not work on complex eigen values
    sigma = broadcast((x -> sqrt(abs(x))), sigma)
    singVec_V = singVec_V[:, sortperm(sigma)]
    sigma = sort(sigma)


    # compute the rank
    if cutNum == -1
        # compute the nonzero singular values
        sig_rank = size(filter((x -> x != 0), sigma), 1)
    else
        sig_rank = cutNum
    end

    # only remain the part we need
    sigma = sigma[1:sig_rank]

    # Compute left singular vectors
    # initializing the left singular vector
    singVec_U = Array{Float64}(undef, m, sig_rank)
        
    # Compute the left singular vector
    for i in 1:sig_rank
        # compute AV_i
        temp_A = A * singVec_V[:, i]
        # normalize AV_i
        temp_A_normal = temp_A / sigma[i]
        singVec_U[:,i] = temp_A_normal'
    end
    if !tall
        return singVec_V, sigma, singVec_U
    else
        return singVec_U, sigma, singVec_V
    end
end

mySVD (generic function with 2 methods)

In [26]:
A = rand(5,6)
myF = mySVD(A);
myU, myS, myV = myF;
F = svd(A)
U, S, V = F
A ≈ myU * Diagonal(myS) * myV'

true

In [18]:
norm(A - U * Diagonal(S) * V') / norm(A)

4.0042633357314872e-16

In [19]:
norm(A - myU * Diagonal(myS) * myV') / norm(A)

5.279987673343016e-16

In [20]:
norm(myU' * myU - I)

3.674699973122493e-14

In [21]:
norm(myU * myU' - I)

1.0

In [15]:
norm(myV' * myV - I)

9.408982134980724e-16